### Importacion de librerias


In [2]:
import sys
import os
import pandas as pd
import numpy as np
from psycopg2 import sql
from sqlalchemy import text
sys.path.append(os.path.abspath('../source'))
from connection_db.db_utils import get_connection

### Conexión a la base de datos



Este fragmento de código establece una conexión a una base de datos PostgreSQL utilizando la función "get_connection" del módulo "connection_db.db_utils", lee todos los datos de la tabla "accidents" en un DataFrame de Pandas, muestra las primeras filas del DataFrame.

In [3]:
engine = get_connection()

df = pd.read_sql_query("SELECT * FROM accidents", engine)

df.head()


Engine creado exitosamente


,id,country,year,month,day_of_week,time_of_day,urban_rural,road_type,weather_conditions,visibility_level,...,number_of_fatalities,emergency_response_time,traffic_volume,road_condition,accident_cause,insurance_claims,medical_cost,economic_loss,region,population_density
0,1,USA,2002,October,Tuesday,Evening,Rural,Street,Windy,220.414651,...,2,58.625720,7412.752760,Wet,Weather,4,40499.856982,22072.878502,Europe,3866.273014
1,2,UK,2014,December,Saturday,Evening,Urban,Street,Windy,168.311358,...,1,58.041380,4458.628820,Snow-covered,Mechanical Failure,3,6486.600073,9534.399441,North America,2333.916224
2,3,USA,2012,July,Sunday,Afternoon,Urban,Highway,Snowy,341.286506,...,4,42.374452,9856.915064,Wet,Speeding,4,29164.412982,58009.145124,South America,4408.889129
3,4,UK,2017,May,Saturday,Evening,Urban,Main Road,Clear,489.384536,...,3,48.554014,4958.646267,Icy,Distracted Driving,3,25797.212566,20907.151302,Australia,2810.822423
4,5,Canada,2002,July,Tuesday,Afternoon,Rural,Highway,Rainy,348.344850,...,4,18.318250,3843.191463,Icy,Distracted Driving,8,15605.293921,13584.060759,South America,3883.645634


# Transformación 

### Eliminación de columnas redundantes

Este código utiliza la función `unique()` de Pandas para identificar y mostrar los valores únicos presentes en las columnas "country" y "region" del DataFrame `df`. Primero, imprime los valores únicos de la columna "country", seguido de un salto de línea y los valores únicos de la columna "region". Esto permite inspeccionar rápidamente las categorías o valores distintos presentes en estas columnas, lo cual es útil para comprender la distribución de los datos de estás columnas.

In [4]:
print("Valores únicos en la columna 'country':")
print(df["country"].unique())

print("\nValores únicos en la columna 'region':")
print(df["region"].unique())

Valores únicos en la columna 'country':
['USA' 'UK' 'Canada' 'India' 'China' 'Japan' 'Russia' 'Brazil' 'Germany'
 'Australia']

Valores únicos en la columna 'region':
['Europe' 'North America' 'South America' 'Australia' 'Asia']


In [5]:
df = df.drop(columns=["region"])

La columna "region" ha sido eliminada del dataset, ya que no aporta información relevante para los objetivos del proyecto. Dado que ya contamos con la columna "country", la cual especifica el país donde ocurrió cada accidente, la variable "region" se vuelve redundante. Además, el nivel de detalle que proporciona la columna de país es más preciso y útil para el análisis, mientras que la región o continente es una categorización más amplia que no aporta valor significativo a nuestro estudio. 

### Conformidad 

#### Número de cifras decimales

El código examina las columnas con tipo de datos 'float' en DataFrame `df` determinando el número máximo de decimales presentes en cada columna para comprender la precisión de los datos. Luego, redondea todos los valores de estas columnas a dos decimales, simplificando los datos y mejorando su legibilidad para análisis y presentación.

In [6]:
columnas_float = df.select_dtypes(include=['float'])


for col in columnas_float.columns:
    max_decimales = columnas_float[col].astype(str).str.split('.').str[1].str.len().max()
    print(f"Número máximo de decimales en la columna '{col}': {max_decimales}")


Número máximo de decimales en la columna 'visibility_level': 15
Número máximo de decimales en la columna 'driver_alcohol_level': 20
Número máximo de decimales en la columna 'emergency_response_time': 16
Número máximo de decimales en la columna 'traffic_volume': 14
Número máximo de decimales en la columna 'medical_cost': 14
Número máximo de decimales en la columna 'economic_loss': 13
Número máximo de decimales en la columna 'population_density': 15


In [7]:
columnas_float_redondeadas = columnas_float.round(2)

print(columnas_float_redondeadas.head())

   visibility_level  driver_alcohol_level  emergency_response_time  \
0            220.41                  0.05                    58.63   
1            168.31                  0.23                    58.04   
2            341.29                  0.14                    42.37   
3            489.38                  0.12                    48.55   
4            348.34                  0.16                    18.32   

   traffic_volume  medical_cost  economic_loss  population_density  
0         7412.75      40499.86       22072.88             3866.27  
1         4458.63       6486.60        9534.40             2333.92  
2         9856.92      29164.41       58009.15             4408.89  
3         4958.65      25797.21       20907.15             2810.82  
4         3843.19      15605.29       13584.06             3883.65  


#### Valores de la columna 'driver_alcohol_level'

Los niveles de alcohol en la sangre pueden ser agrupados en categorías significativas (Bajo, Moderado, Alto, etc.) que representan diferentes niveles de riesgo o impacto, en lugar de tratarse como simples valores continuos difíceles de interpretar para el público general. Para esto se utiliza el "Binning", que  implica convertir una variable numérica continua en una variable categórica ordinal al agrupar los valores en intervalos o "bins".

Este código define una función llamada `categorizar_alcohol_level` que clasifica los niveles de alcohol en la sangre (driver_alcohol_level) en categorías como "Bajo", "Moderado", "Alto", "Peligroso" y "Letal", basándose en umbrales específicos. Luego, aplica esta función a la columna "driver_alcohol_level" del DataFrame `df` para crear una nueva columna llamada "Alcohol_Level_Category", que contiene las categorías correspondientes para cada valor de nivel de alcohol. Finalmente, imprime la columna "Alcohol_Level_Category" para mostrar las clasificaciones resultantes. 

In [8]:
def categorizar_alcohol_level(driver_alcohol_level):
    if driver_alcohol_level < 0.03:
        return "Bajo"
    elif driver_alcohol_level < 0.08:
        return "Moderado"
    elif driver_alcohol_level < 0.20:
        return "Alto"
    elif driver_alcohol_level < 0.30:
        return "Peligroso"
    else:
        return "Letal"

df["Alcohol_Level_Category"] = df["driver_alcohol_level"].apply(categorizar_alcohol_level)

print(df["Alcohol_Level_Category"])


0          Moderado
1         Peligroso
2              Alto
3              Alto
4              Alto
            ...    
131995     Moderado
131996     Moderado
131997     Moderado
131998         Bajo
131999         Alto
Name: Alcohol_Level_Category, Length: 132000, dtype: object


Esta conversión no solo optimiza la representación de los datos en visualizaciones, sino que también permite la identificación de patrones clave entre los niveles de alcohol y otros factores críticos, como la severidad de los accidentes o el número de víctimas. 


#### Valores de la columna 'visibility_level'

La función `categorize_visibility` reemplaza la columna "visibility_level" por "Visibility_Category", la cual almacena la clasificación correspondiente para cada registro, pasando de un valor numérico a una categoría descriptiva: Muy Baja, Baja, Moderada o Alta. Esto permite transformar una variable numérica en categórica, facilitando la interpretación de los datos y su representación en visualizaciones.


In [9]:
def categorize_visibility(visibility_level):
    if visibility_level < 200:
        return "Muy Baja"
    elif visibility_level < 300:
        return "Baja"
    elif visibility_level < 400:
        return "Moderada"
    else:
        return "Alta"

df["Visibility_Category"] = df["visibility_level"].apply(categorize_visibility)

print(df["Visibility_Category"])

0             Baja
1         Muy Baja
2         Moderada
3             Alta
4         Moderada
            ...   
131995    Moderada
131996    Muy Baja
131997    Muy Baja
131998        Baja
131999        Baja
Name: Visibility_Category, Length: 132000, dtype: object


#### Valores de "days_order", "months_order" y "time_of_day"

In [10]:
print("\nValores únicos en la columna 'time_of_day':")
print(df["time_of_day"].unique())


Valores únicos en la columna 'time_of_day':
['Evening' 'Afternoon' 'Night' 'Morning']


Las columnas "day_of_week", "month" y "time, of day" son variables categóricas ordinales, lo que significa que las categorías tienen un orden lógico (los días de la semana y los meses del año tienen un orden específico).Al estructurar las variables con un orden definido, se facilita la representación gráfica en gráficos de tendencia o análisis estacionales, evitando errores en la disposición de los datos. Asimismo, esta verificación contribuye a la correcta interpretación de patrones temporales en la ocurrencia de accidentes, lo que puede ser clave para la toma de decisiones en seguridad vial y planificación de estrategias preventivas.

Este código transforma las columnas categóricas "day_of_week", "month" y "time_of_day" en un tipo de dato categórico ordenado en Pandas. Esto permite que el DataFrame conozca el orden lógico de los días de la semana, los meses del año, y el tiempo del día. Se utiliza `pd.Categorical directamente, que es más eficiente y conciso que pd.CategoricalDtype y .astype(). Además, se ha simplificado la impresión de los resultados, eliminando la necesidad de verificar si las columnas están ordenadas, ya que pd.Categorical con ordered=True garantiza que lo estén. Esto facilita el análisis de datos temporales donde el orden es crucial.


In [11]:
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
time_of_day_order = ["Morning", "Afternoon", "Evening", "Night"]


df["day_of_week"] = pd.Categorical(df["day_of_week"], categories=days_order, ordered=True)
df["month"] = pd.Categorical(df["month"], categories=months_order, ordered=True)
df["time_of_day"] = pd.Categorical(df["time_of_day"], categories=time_of_day_order, ordered=True)

print("Día de semana (Categórico):\n", df["day_of_week"].head(5))
print("\nMes (Categórico):\n", df["month"].head(5))
print("\nTiempo del día (Categórico):\n", df["time_of_day"].head(5))


Día de semana (Categórico):
 0     Tuesday
1    Saturday
2      Sunday
3    Saturday
4     Tuesday
Name: day_of_week, dtype: category
Categories (7, object): ['Monday' < 'Tuesday' < 'Wednesday' < 'Thursday' < 'Friday' < 'Saturday' < 'Sunday']

Mes (Categórico):
 0     October
1    December
2        July
3         May
4        July
Name: month, dtype: category
Categories (12, object): ['January' < 'February' < 'March' < 'April' ... 'September' < 'October' < 'November' < 'December']

Tiempo del día (Categórico):
 0      Evening
1      Evening
2    Afternoon
3      Evening
4    Afternoon
Name: time_of_day, dtype: category
Categories (4, object): ['Morning' < 'Afternoon' < 'Evening' < 'Night']


#### Valores únicos de otras columnas con dtype String

Con el el método `.unique()` de pandas se identifican y extraen los valores únicos presentes en las columnas driver_fatigue,vehicle_condition, accident_severity, road_type, weather_conditions, driver_gender y driver_age_group.

In [16]:
print("\nValores únicos en la columna 'driver_fatigue':")
print(df["driver_fatigue"].unique())

print("\nValores únicos en la columna 'vehicle_condition':")
print(df["vehicle_condition"].unique())

print("\nValores únicos en la columna 'accident_severity':")
print(df["accident_severity"].unique())

print("\nValores únicos en la columna 'road_type':")
print(df["road_type"].unique())

print("\nValores únicos en la columna 'weather_conditions':")
print(df["weather_conditions"].unique())

print("\nValores únicos en la columna 'driver_gender':")
print(df["driver_gender"].unique())

print("\nValores únicos en la columna 'driver_age group':")
print(df["driver_age_group"].unique())


Valores únicos en la columna 'driver_fatigue':
[False  True]

Valores únicos en la columna 'vehicle_condition':
['Poor' 'Moderate' 'Good']

Valores únicos en la columna 'accident_severity':
['Moderate' 'Minor' 'Severe']

Valores únicos en la columna 'road_type':
['Street' 'Highway' 'Main Road']

Valores únicos en la columna 'weather_conditions':
['Windy' 'Snowy' 'Clear' 'Rainy' 'Foggy']

Valores únicos en la columna 'driver_gender':
['Male' 'Female']

Valores únicos en la columna 'driver_age group':
['18-25' '41-60' '26-40' '<18' '61+']


Si hubiera variaciones en la forma en que se representan los datos (por ejemplo, "Male", "male", "M"), sería necesario estandarizar los valores. En este caso no hay variaciones en la forma en que se representan los datos. Sin embargo, la columna "driver_fatigue" presenta valores binarios (0,1) por lo que es óptimo convertirla a dtype booleano usando `.astype(bool)`.

In [13]:
df["driver_fatigue"] = df["driver_fatigue"].astype(bool)

print(df["driver_fatigue"].head())

0    False
1     True
2    False
3     True
4     True
Name: driver_fatigue, dtype: bool


# Carga

In [14]:
try:
    df.to_sql('accidents_clean', engine, index=False)
    print("Datos guardados exitosamente en la tabla 'accidents_clean'")
except Exception as e:
    print(f"Error al guardar los datos: {e}")

Datos guardados exitosamente en la tabla 'accidents_clean'
